In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(0)

## Load Model

In [4]:
name = 'baseline_any_xresnet18_weighted'

In [5]:
def get_data(bs, sz, img_dir=path_jpg256, weighted=True):
    wgts = get_wgts(Meta.df_comb, Meta.splits) if weighted else None
    return get_data_gen(L(list(Meta.df_comb.index)), bs=bs, img_tfm=get_pil_fn(path/img_dir), 
                        sz=sz, splits=Meta.splits, wgts=wgts)

In [6]:
dbch = get_data(512, 128)
learn = get_learner(dbch, xresnet18)

## Training

In [ ]:
learn.lr_find()

In [7]:
do_fit(learn, 20, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.240352,0.161903,0.951774,0.881066,06:42
1,0.209411,0.174868,0.945869,0.866698,06:30
2,0.198383,0.130193,0.961702,0.912259,06:29
3,0.191036,0.111996,0.965896,0.927127,07:33
4,0.186231,0.110861,0.966653,0.928265,07:03
5,0.178425,0.180665,0.943509,0.834867,06:31
6,0.174824,0.123761,0.960611,0.908473,07:02
7,0.171438,0.101200,0.968883,0.937614,07:29
8,0.171310,0.108721,0.965827,0.924798,07:39
9,0.160609,0.129294,0.960343,0.903592,07:17


In [8]:
learn.dbunch = get_data(512, 256)
learn.load(f'runs/{name}-1')

In [ ]:
learn.lr_find()

In [9]:
do_fit(learn, 20, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.153996,0.103929,0.968087,0.930882,13:58
1,0.122456,0.099257,0.969615,0.936773,13:49
2,0.105267,0.105466,0.968791,0.935763,13:58
3,0.097388,0.115915,0.965638,0.925426,13:50
4,0.090658,0.107752,0.970977,0.943187,13:47
5,0.083660,0.109115,0.969030,0.937156,13:47
6,0.075790,0.108416,0.970150,0.940836,13:48
7,0.070731,0.118083,0.967804,0.932913,13:47
8,0.065276,0.111773,0.970718,0.943027,13:48
9,0.056352,0.121967,0.970643,0.942197,13:44


In [ ]:
THIS IS WRONG. SHOULD BE get_data(256, 512)
learn.dbunch = get_data(512, 256, weighted=False)
do_fit(learn, 2, 5e-4)
learn.save(f'runs/{name}-3-full')

## Submission

In [6]:
learn.load(f'runs/{name}-3-full')
sub_fn = f'subm/{name}'

In [8]:
learn.dbunch = get_test_data(Meta.df_tst, bs=512, sz=None, tst_dir='tst_jpg256')

In [9]:
ob = learn.dbunch.one_batch()

In [10]:
preds,targs = learn.get_preds()

In [11]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [12]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 10.0MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [13]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13768428,
 'totalBytes': 27277209,
 'date': '2019-12-12T18:13:15.577Z',
 'description': 'baseline_any_xresnet18',
 'errorDescription': None,
 'fileName': 'baseline_any_xresnet18.csv',
 'publicScore': '0.78783',
 'privateScore': '0.07677',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13768428/13768428.raw'}